# Feature Extraction using VGG

One way to extract features is by using CNNs. For this purpose we will use VGG16.

The CNN should be used as fixed feature extractor on a new task. 

## Libraries

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import tensorflow as tf
from keras.layers import *
import keras

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2
import os

import itertools

## Dataset

The dataset is taken by Kaggle at the following link: https://www.kaggle.com/slothkong/10-monkey-species .\
It's called "10 Monkey Species" and as the name suggest the task is to classify 10 species of monkey from images. \
The dataset is balanced. \


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = '/content/drive/MyDrive'
base_dir = root_dir + '/AML - Assigment 4/10 Monkey Species'


Mounted at /content/drive


In [4]:
train_path = base_dir + '/training/'
test_path = base_dir + '/validation/'

Now we just try to read an image to see how the images are.

In [9]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

## Models

In [38]:
#This give us all the model but the last layer
from tensorflow.keras.models import Model
base_model = VGG16(weights='imagenet')
base_model.summary()

553476096/553467096 [==============================] - 6s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

If we apply multiple cut we have multiple features. \
We have to find where to cut to extract usefull information.

To better manage the RAM there are various options:
If I make a network with input 24x24 and put 12x12 inputs may break the convolutionary layers because they are thought differently.

1.   Resize in memory often can't be done
2.   Upsample/Downsample before putting it in the NN
3.   Upsample/Downsample as first layer of the NN (not trainable, best option).



Since my dataset is not big I can pre-process it witout problems. \
Anyway, I can't use a MaxPooling Layer because the inputs have different shapes. \


In [39]:
model_1 = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output) # block4_pool

In [40]:
model_2 = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

In [41]:
model_3 = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

I have chosen the layers so that they are more and more specific for the initial task. \
Unfortunately I was unable to get higher layer output due to colab related memory problems. \
However, I am convinced that layers higher up, but not too much, can help more.

### Model 1

In [42]:
test_features_1 = []
for image in flatted_test:
  test_features_1.append( model_1.predict(image) )

print(len(test_features_1))

272


In [43]:
train_features_1 = []
for image in flatted_train:
  train_features_1.append( model_1.predict(image) )

print(len(train_features_1))

1098


### Model 2

In [44]:
test_features_2 = []
for image in flatted_test:
  test_features_2.append( model_2.predict(image) )

print(len(test_features_2))

272


In [45]:
train_features_2 = []
for image in flatted_train:
  train_features_2.append( model_2.predict(image) )

print(len(train_features_2))

1098


### Model 3

In [46]:
test_features_3 = []
for image in flatted_test:
  test_features_3.append( model_3.predict(image) )

print(len(test_features_3))

272


In [47]:
train_features_3 = []
for image in flatted_train:
  train_features_3.append( model_3.predict(image) )

print(len(train_features_3))

1098
